In [ ]:
!pip install earthengine-api geemap --upgrade

In [1]:
import ee

ee.Initialize(project="ee-romdijoux")

# Collect Vertical Transmit-Vertical Receive (VV) and Vertical Transmit-Horizontal Receive (VH) bands from Sentinel-1 at 10m resolution.
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD#description
sentinel1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH"))
    .filter(ee.Filter.eq("resolution_meters", 10))
    .select(["VV", "VH"])
)

# Load Sentinel-2 Level 1C data, only RGB bands (B4, B3, and B2) captured at 10m resolution
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2#description
sentinel2 = ee.ImageCollection("COPERNICUS/S2").select(["B4", "B3", "B2"])


In [2]:
def sampleSentinelData(location, date):
    start_date = date.advance(-15, "days")
    end_date = date.advance(15, "days")

    sentinel1_image = (
        sentinel1.filterDate(start_date, end_date).filterBounds(location).first()
    )
    sentinel2_image = (
        sentinel2.filterDate(start_date, end_date)
        .filterBounds(sentinel1_image.geometry().centroid())
        .first()
    )

    sentinel1_image_clipped = sentinel1_image.clip(sentinel2_image.geometry())
    sentinel2_image_clipped = sentinel2_image.clip(sentinel1_image_clipped.geometry())

    return (sentinel1_image_clipped, sentinel2_image_clipped)

In [3]:
from datetime import datetime, timedelta


## TODO: add more location, this is returing locations mostly in France but can be in the ocean or in neighboring countries too
def sampleCoord():
    return [random.uniform(-5.0, 9.0), random.uniform(42.0, 51.0)]


def sampleDate(start_date, end_date):
    start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
    end_datetime = datetime.strptime(end_date, "%Y-%m-%d")
    date_range = (end_datetime - start_datetime).days
    random_days = random.randint(0, date_range)
    result_date = start_datetime + timedelta(days=random_days)
    return result_date.strftime("%Y-%m-%d")

In [27]:
import random
import geemap

start_date = "2018-01-01"
end_date = "2022-01-01"

# Generate a random location within ~ France
location  = sampleCoord()
date = sampleDate(start_date, end_date)

(sentinel1_image, sentinel2_image) = sampleSentinelData(
    ee.Geometry.Point(location), ee.Date(date)
)

# rectangle of ~ 512x512
bounds = sentinel1_image.geometry().centroid().buffer(distance=2560).bounds()

sentinel1_image = sentinel1_image.clip(bounds)
sentinel2_image = sentinel2_image.clip(bounds)

print("Sentinel-1 Image:", sentinel1_image.getInfo())
print("Sentinel-2 Image:", sentinel2_image.getInfo())

cloud_percent = sentinel2_image.getInfo()["properties"]["CLOUD_COVERAGE_ASSESSMENT"]
print(cloud_percent, "percents of cloud coverage")

m = geemap.Map()
m.centerObject(sentinel1_image)
m.add_layer(sentinel1_image, {"min": -25, "max": 5}, "Sentinel-1", True)
m.add_layer(
    sentinel2_image,
    {"bands": ["B4", "B3", "B2"], "min": 0, "max": 65535, "gamma": 4.0},
    "Sentinel-2",
    True,
)

m

Sentinel-1 Image: {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [515, 517], 'origin': [9518, 15065], 'crs': 'EPSG:32630', 'crs_transform': [10, 0, 355218.1893714958, 0, -10, 5205823.707739933]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [515, 517], 'origin': [9518, 15065], 'crs': 'EPSG:32630', 'crs_transform': [10, 0, 355218.1893714958, 0, -10, 5205823.707739933]}], 'version': 1704698728146265, 'id': 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20180301T180417_20180301T180442_020825_023B56_C843', 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[-3.6359553100265773, 45.60222375608424], [-3.570506614830064, 45.602223756084236], [-3.570506614830064, 45.64828910041608], [-3.6359553100265773, 45.64828910041608], [-3.6359553100265773, 45.60222375608424]]]}, 'SNAP_Graph_Processing_Framework_GPF_vers': '6.0.4', 'SLC_Processing_facility_org': 'ESA', 'SLC_Processing_faci

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [29]:
# Export a bunch of samples to drive

start_date = "2018-01-01"
end_date = "2022-01-01"

# can be dangerous to do more
for i in range(100):
    # Generate a random location within ~ France
    location  = sampleCoord()
    date = sampleDate(start_date, end_date)

    (sentinel1_image, sentinel2_image) = sampleSentinelData(
        ee.Geometry.Point(location), ee.Date(date)
    )

    sentinel1_image_info = sentinel1_image.getInfo()
    sentinel2_image_info = sentinel2_image.getInfo()
    # rectangle of ~ 512x512 (~2 to 14px differences)
    roi = sentinel1_image.geometry().centroid().buffer(distance=2560).bounds()
    cloud_coverage = sentinel2_image_info["properties"]["CLOUD_COVERAGE_ASSESSMENT"]

    description = "cloud coverage: " + str(cloud_coverage)

    task = ee.batch.Export.image.toDrive(
        image=sentinel1_image,
        description='image_export'+ str(i),
        folder='Sentinel1_samples',
        fileNamePrefix= str(i) + '_sentinel1_'+ date + '_lat_' + str(location[0]) + '_long_' + str(location[1]),
        region=roi,
        dimensions='512x512',
        crs='EPSG:32630'
        )
    task.start()

    task = ee.batch.Export.image.toDrive(
        image=sentinel2_image,
        description='image_export'+ str(i),
        folder='Sentinel2_samples',
        fileNamePrefix= str(i) + '_sentinel2_'+ date + '_lat_' + str(location[0]) + '_long_' + str(location[1]) + '_cloud_coverage_' + str(cloud_coverage),
        region=roi,
        #scale=10, #resolution in meter per pixel
        dimensions='512x512',
        crs='EPSG:32630'
        )
    task.start()
